In [41]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as plt 

In [3]:
pd.options.display.max_rows = 4000
pd.options.display.max_seq_items = 2000
pd.set_option('display.max_colwidth', -1)

In [4]:
ed_country = pd.read_csv("~\Desktop\dss_sp19\EdStatsCountry.csv")

In [5]:
ed_data = pd.read_csv("~\Desktop\dss_sp19\EdStatsData.csv")

In [6]:
region_count = ed_country.groupby("Region").size()
region_count

Region
East Asia & Pacific           36
Europe & Central Asia         57
Latin America & Caribbean     41
Middle East & North Africa    21
North America                 3 
South Asia                    8 
Sub-Saharan Africa            48
dtype: int64

In [127]:
by_region = ed_country.sort_values("Region")[["Country Code", "Short Name", "Region", "Income Group"]]
only_east_asia = by_region[by_region["Region"] == "East Asia & Pacific"]

In [142]:
countries_to_2017 = ed_data.iloc[:, :52] #table went all the way to 2100, so I took out the years that have not occurred yet
countries_to_2017.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1970,1971,1972,1973,1974,1975,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,Arab World,ARB,"Adjusted net enrolment rate, lower secondary, both sexes (%)",UIS.NERA.2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Arab World,ARB,"Adjusted net enrolment rate, lower secondary, female (%)",UIS.NERA.2.F,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arab World,ARB,"Adjusted net enrolment rate, lower secondary, gender parity index (GPI)",UIS.NERA.2.GPI,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arab World,ARB,"Adjusted net enrolment rate, lower secondary, male (%)",UIS.NERA.2.M,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arab World,ARB,"Adjusted net enrolment rate, primary, both sexes (%)",SE.PRM.TENR,54.822121,54.894138,56.209438,57.267109,57.991138,59.36554,...,84.011871,84.195961,85.211998,85.24514,86.101669,85.51194,85.320152,NaN,NaN,NaN


In [125]:
us_data = countries_to_2017[countries_to_2017['Country Code'] == 'USA']
us_data = us_data.drop(columns=["Country Name", "Country Code", "Indicator Code"]) #without "U.S." label
us_row_counts = us_data.apply(lambda x: x.count(), axis=1) #number of data points in row
us_data['Row Count'] = us_row_counts
us_data = us_data[us_data['Row Count'] > 40] #rows with more than 40 data points

In [141]:
china_data = countries_to_2017[countries_to_2017['Country Code'] == 'CHN']
china_data = china_data.drop(columns=["Country Name", "Country Code", "Indicator Code"]) #without "China" label
china_row_counts = china_data.apply(lambda x: x.count(), axis=1)
china_data['Row Count'] = china_row_counts
china_data = china_data[china_data['Row Count'] > 40]

In [116]:
china_primary_enroll = china_data[china_data['Indicator Name'] == 'Enrolment in primary education, both sexes (number)'].transpose()
#transposed to make the rows years

china_primary_enroll = china_primary_enroll.rename(columns={243073: "Enrollment in primary education, both sexes (number)"})
#renamed column

china_primary_enroll = china_primary_enroll.drop(["Indicator Name"])
#dropped row with indicator name

In [112]:
china_secondary_enroll = china_data[china_data['Indicator Name'] == 'Enrolment in secondary education, both sexes (number)'].transpose()
china_secondary_enroll = china_secondary_enroll.rename(columns={243081: "Enrollment in secondary education, both sexes (number)"})
china_secondary_enroll = china_secondary_enroll.drop(["Indicator Name"])

In [110]:
china_tertiary_enroll = china_data[china_data['Indicator Name'] == 'Enrolment in tertiary education, all programmes, both sexes (number)'].transpose()
china_tertiary_enroll = china_tertiary_enroll.rename(columns={243094: "Enrollment in tertiary education, all programmes, both sexes (number)"})
china_tertiary_enroll = china_tertiary_enroll.drop(["Indicator Name"])

In [128]:
china_primary_secondary = china_primary_enroll.merge(china_secondary_enroll, left_index=True, right_index=True)
china_enroll_all_levels = china_primary_secondary.merge(china_tertiary_enroll, left_index=True, right_index=True)
china_enroll_all_levels.head()

,"Enrollment in primary education, both sexes (number)","Enrollment in secondary education, both sexes (number)","Enrollment in tertiary education, all programmes, both sexes (number)"
1970,1.00668e+08,2.02534e+07,108617
1971,1.0528e+08,2.64825e+07,NaN
1972,1.12112e+08,3.14937e+07,NaN
1973,1.25492e+08,3.61669e+07,193719
1974,1.35704e+08,3.49468e+07,313645


In [135]:
china_gdp_per_capita = china_data[china_data['Indicator Name'] == 'GDP per capita (current US$)'].transpose()
china_gdp_per_capita = china_gdp_per_capita.rename(columns={243135: "GDP per capita (current US$)"})
china_gdp_per_capita = china_gdp_per_capita.drop(["Indicator Name"])

In [138]:
china_enrollment_and_gdp = china_gdp_per_capita.merge(china_enroll_all_levels, left_index=True, right_index=True)
china_enrollment_and_gdp.head()

,GDP per capita (current US$),"Enrollment in primary education, both sexes (number)","Enrollment in secondary education, both sexes (number)","Enrollment in tertiary education, all programmes, both sexes (number)"
1970,113.163,1.00668e+08,2.02534e+07,108617
1971,118.655,1.0528e+08,2.64825e+07,NaN
1972,131.884,1.12112e+08,3.14937e+07,NaN
1973,157.09,1.25492e+08,3.61669e+07,193719
1974,160.14,1.35704e+08,3.49468e+07,313645


In [139]:
china_primary_male_life_expect = china_data[china_data['Indicator Name'] == 'School life expectancy, primary, male (years)'].transpose()
china_primary_female_life_expect = china_data[china_data['Indicator Name'] == 'School life expectancy, primary, female (years)'].transpose()